In [1]:
import requests as req
import pandas as pd

```
import asyncio

from chessdotcom.aio import get_player_profile, Client
#or
from chessdotcom import Client
Client.aio = True

usernames = ["fabianocaruana", "GMHikaruOnTwitch", "MagnusCarlsen", "GarryKasparov"]

cors = [get_player_profile(name) for name in usernames]

async def gather_cors(cors):
   responses = await asyncio.gather(*cors)
   return responses

responses = asyncio.run(gather_cors(cors))
```

In [2]:
headers = {
    "User-Agent":"duduardo45"
}

users = {
    'MrBarros': {
        'url':'https://api.chess.com/pub/player/mrbarros'
    },
}

ratings = {}

# api_url = "https://api.chess.com/pub/player/"

In [3]:
def get(s):
    return req.get(s, headers = headers)

# acessa os {jogador}/games/archives, pega todos os links

In [4]:
archives = "/games/archives"

u = list(users.keys())[0]

response = get(f'{users[u]['url']}{archives}')
datas = response.json()
datas = datas['archives']

# acessa cada um dos links e baixa o pgn de resposta de todos, acumulando ele em um único pgn por usuario ao terminar muda o valor no dicionario de users para True

In [5]:
player_games = []

for data in datas:
    response = get(data)
    monthly_games = response.json()['games']
    player_games.extend(monthly_games)

player_games = pd.DataFrame(player_games)
player_games['player'] = u

In [6]:
print(player_games.iloc[-1]['white'].keys())

dict_keys(['rating', 'result', '@id', 'username', 'uuid'])


In [ ]:
def extract_black_white(row):
    white = row['white']
    black = row['black']
    wplayer = white['username']
    bplayer = black['username']
    wresult = white['result']
    bresult = black['result']
    wrating = white['rating']
    brating = black['rating']
    if wplayer == row['player']:
        users[bplayer]['url'] = black['@id']
        prating = wrating
        opponent = bplayer
        orating = brating
    elif bplayer == row['player']:
        users[wplayer]['url'] = white['@id']
        prating = brating
        opponent = wplayer
        orating = wrating
    if wresult == 'win':
        winner = wplayer
        method = bresult
    elif bresult == 'win':
        winner = bplayer
        method = wresult
    

    
    # pensar em extrair @id dos jogadores para ter já o link para perfis diferentes

    
    


In [7]:
player_games['white player'] = player_games['white'].apply(lambda row: row['username'] if ('username' in row.keys()) else pd.NA)
player_games['black player'] = player_games['black'].apply(lambda row: row['username'] if ('username' in row.keys()) else pd.NA)

player_games['opponent'] = player_games.loc[player_games['white player'].str.lower() != u, 'white player']
player_games.loc[player_games['opponent'].isna(),'opponent'] = player_games.loc[player_games['black player'].str.lower() != u, 'black player']

# recupera todos os usernames unicos contra o quem o jogador jogou, salva eles no dicionario com valor False, para registrar que não foi pego os arquivos deste user

# procura algum usuario no dicionario para o qual não foi feito o processo e repete para este

# adiciona o usuario ao json users.json e salva a alteração

### falta decidir o que fazer com o pgn de cada usuário; considerar adicionar o uuid de cada jogo para não salvar o dobro de dados